In [13]:
### IMPORTING FILES WITH NATIVE LIB ###
import csv

with open("csv/currriculos.csv", "r") as cur:
    cur_csv = csv.reader(cur, delimiter=",")
    for l in cur_csv:
        print(l)

In [31]:
### IMPORTING FILES WITH PANDAS ###
import pandas as pd

cur = pd.read_csv("csv/currriculos.csv", sep=",")
display(cur)

(4, 4)

In [17]:
## APLICAÇÃO DO ALGORITMO BAG OF WORDS + RANQUEAMENTO UTILIZANDO PALAVRAS CHAVE ##
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

# Seus dados fictícios
data = pd.read_csv("csv/currriculos.csv", sep=",")


# Criar um DataFrame a partir dos dados fictícios
df = pd.DataFrame(data)

# Conhecimentos, Habilidades e Atitudes
conhecimentos = ["python", "graduação", "estruturas de dados", "desenvolvimento ágil", "bancos de dados"]
habilidades = ["design escalável", "depuração", "integração de APIs", "testes de unidade", "comunicação", "liderança"]
atitudes = ["comprometimento", "aprendizado contínuo", "adaptabilidade", "trabalho em equipe", "ética"]
cha = conhecimentos + habilidades + atitudes

# Inicializar o vetorizador Bag of Words
vectorizer = CountVectorizer()

# Inicializar as colunas de pontuação
df["Pontuacao_Conhecimentos"] = 0
df["Pontuacao_Habilidades"] = 0
df["Pontuacao_Atitudes"] = 0

# Percorrer cada palavra-chave de conhecimento, habilidade e atitude
for palavra_chave in conhecimentos + habilidades + atitudes:
    # Verificar se a palavra-chave está presente nas experiências dos candidatos
    df[palavra_chave] = df["Experiencia"].str.contains(palavra_chave, case=False, regex=True).astype(int)
    
    # Atualizar a pontuação correspondente para cada candidato
    if palavra_chave in conhecimentos:
        df["Pontuacao_Conhecimentos"] += df[palavra_chave]
    elif palavra_chave in habilidades:
        df["Pontuacao_Habilidades"] += df[palavra_chave]
    elif palavra_chave in atitudes:
        df["Pontuacao_Atitudes"] += df[palavra_chave]

# Calcular a pontuação total para cada candidato
df["Classificacao"] = df["Pontuacao_Conhecimentos"] + df["Pontuacao_Habilidades"] + df["Pontuacao_Atitudes"]

# Classificar os candidatos por pontuação total em ordem decrescente
df_classificado = df.sort_values(by="Classificacao", ascending=False)

# Salvar o resultado em um arquivo CSV com todas as informações
df_classificado.to_csv("csv/results/1_curriculos_classificados_com_critérios.csv", index=False)

# Criar um DataFrame com a quantidade total de conhecimento, habilidade e atitude de cada pessoa
df_total = df[["Nome", "Pontuacao_Conhecimentos", "Pontuacao_Habilidades", "Pontuacao_Atitudes", "Classificacao"]].sort_values(by="Classificacao", ascending=False)

# Salvar o resultado em um arquivo CSV separado
df_total.to_csv("csv/results/2_quantidade_total_conhecimento_habilidade_atitude.csv", index=False)

In [22]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# Carregar o CSV com as pontuações de conhecimento, habilidade e atitude
df = pd.read_csv("csv/results/1_curriculos_classificados_com_critérios.csv")

# Dividir o conjunto de dados em recursos (X) e rótulos (y)
X = df[["Pontuacao_Conhecimentos", "Pontuacao_Habilidades", "Pontuacao_Atitudes"]]
y = df["Classificacao"]  # Supondo que você tenha uma coluna "Classificacao" no CSV com as classes

# Dividir o conjunto de dados em treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicializar o modelo SVM
svm_model = SVC(kernel='linear', C=1)

# Treinar o modelo SVM nos dados de treinamento
svm_model.fit(X_train, y_train)

# Fazer previsões nos dados de teste
y_pred = svm_model.predict(X_test)

# Avaliar o desempenho do modelo
accuracy = accuracy_score(y_test, y_pred)

# Adicionar a acurácia ao DataFrame
df["Acuracia"] = accuracy

# Classificar o DataFrame pela coluna "Classificacao" em ordem decrescente
df = df.sort_values(by="Classificacao", ascending=False)

# Salvar o DataFrame classificado em um arquivo CSV
df.to_csv("csv/results/classificacao_por_classificacao.csv", index=False)